# 事前調査

全ての学習セットからベクトライザーを生成してTFIDF値を計算後、目的のBotに対応するTFIDFベクトルだけを得る方法を検討します。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_benefitone_conversation.csv',
    'test_daikin_conversation.csv',
    'test_ptna_conversation.csv',
    'test_toyotsu_human_conversation.csv'
]
temp_path = TestTool.copy_testdata_csv(learning_dir, csv_file_names)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_toyotsu_human_conversation.csv]


In [43]:
import numpy as np
import pandas as pd


def get_dataset_names_array(dataset_name, count):
    '''
        データセット名称のリストを生成
    '''
    dataset_name_list = [dataset_name] * count
    dataset_names = np.array(dataset_name_list)
    return dataset_names


'''
    学習セットの全CSVファイルから、
    質問文の配列を生成
'''
questions_all = np.empty(0)
answer_ids_all = np.empty(0)
dataset_names_all = np.empty(0)

for csv_file_path in temp_path:
    '''
        TrainingMessageFromCsv と同様の処理

        CSVファイル--->pandasデータフレームの生成
        (TrainingMessageFromCsv#__build_learning_training_messages)
    '''
    learning_training_messages = pd.read_csv(csv_file_path, encoding='utf-8')
    questions = np.array(learning_training_messages['question'])
    answer_ids = np.array(learning_training_messages['answer_id'])
    dataset_names = get_dataset_names_array(os.path.basename(csv_file_path), learning_training_messages.shape[0])

    questions_all = np.concatenate((questions_all, questions))
    answer_ids_all = np.concatenate((answer_ids_all, answer_ids))
    dataset_names_all = np.concatenate((dataset_names_all, dataset_names))

In [45]:
'''
    TextArray#__init__ と同様の処理
'''
from learning.core.nlang import Nlang
from sklearn.feature_extraction.text import TfidfVectorizer

sentences = np.array(questions_all)
separated_sentences = Nlang.batch_split(sentences)
np.size(separated_sentences)

31180

In [46]:
'''
    TextArray#__build_vectorizer と同様の処理
'''
vectorizer = TfidfVectorizer(use_idf=False, token_pattern=u'(?u)\\b\\w+\\b')
vectorizer.fit(separated_sentences)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w+\\b', tokenizer=None, use_idf=False,
        vocabulary=None)

In [47]:
'''
    TextArray#to_vec と同様の処理
'''
feature_vectors = vectorizer.transform(separated_sentences)
feature_vectors

<31180x3162 sparse matrix of type '<class 'numpy.float64'>'
	with 243448 stored elements in Compressed Sparse Row format>

In [54]:
'''
    目的のBotに対応するデータのインデックスを取得
'''
target_data_set_name = 'test_toyotsu_human_conversation.csv'
target_indices = np.where(dataset_names_all==target_data_set_name)
np.size(target_indices)

5064

In [55]:
target_indices

(array([26116, 26117, 26118, ..., 31177, 31178, 31179]),)

In [56]:
'''
    目的のBotに対応するTFIDF値だけを抽出する
    --->これがBotの学習＋予測に使用されるベクトルになります
'''
target_feature_vectors = feature_vectors[target_indices]
target_feature_vectors

<5064x3162 sparse matrix of type '<class 'numpy.float64'>'
	with 49382 stored elements in Compressed Sparse Row format>